In [14]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import math

def main():
    df = pd.read_excel('proto.xlsx', sheet_name='Sheet1')
    tipsCouponRate = df.columns.tolist()[1]
    treasuryCouponRate = df.columns.tolist()[3]
    
    tipsPrice = df.iloc[0,1]
    treasuryPrice = df.iloc[0,3]
    
    startDate = df.iloc[2,1]
    maturityDate = df.iloc[3,1]
    totalPeriods = df.iloc[1,1]
    tipsFrame = df.iloc[6:-1,[0,3]]
    treasuryFrame = df.iloc[6:-1,[4,5]]
   
    df2 = pd.read_excel('proto.xlsx', sheet_name='Sheet2')
    
    
    years = df2.columns.tolist()[1:15]
    periods = [x * 2 for x in years]
    prices = df2.iloc[1,1:15]
    interpolatedPeriods = np.arange(2, int(totalPeriods) + 1, 1)
    intPolStrips = interp1d(periods, prices, kind='cubic')
    discFact = intPolStrips(interpolatedPeriods)
    discFact = np.insert(discFact, 0,  df2.iloc[1,1])
    discFact = np.insert(discFact, 0,  0)

    # inflation swap thingy
    swapYears = df2.iloc[3,1:17]
    swapPeriods = [x * 2 for x in swapYears]
    swapRates = df2.iloc[5,1:17]
    inflationSwap = []
    for i in range(len(swapPeriods)):
        inflationSwap.append(tipsCouponRate/2 * math.pow((1+swapRates[i]/200), swapPeriods[i]))
    intPolSwap = interp1d(swapPeriods, inflationSwap, kind='cubic')
    intPolSwapPrice = intPolSwap(interpolatedPeriods)
    intPolSwapPrice = np.insert(intPolSwapPrice, 0,  inflationSwap[0])
    intPolSwapPrice = np.insert(intPolSwapPrice, 0,  0)


    finalFrame = pd.DataFrame(columns=['Date', 'Treasury', 'Tips', 'Inflation Swaps', 'Strips', 'Total'])
    stripsSum = 0
    for i in range(0, totalPeriods+1):
        if i == 0 :
            finalFrame.loc[i] = [i,-treasuryPrice,-tipsPrice, 0, -7, 5 ]
        elif i == totalPeriods:
            tipsString  = str(100+tipsCouponRate/2)+'*I'+str(i)
            temp = (100+tipsCouponRate/2)* (intPolSwapPrice[i]/(tipsCouponRate/2))
            swapString = str(temp)+ '-'+ tipsString
            val = (100 + treasuryCouponRate/2) -  temp
            stripsSum = stripsSum + val*discFact[i]
            finalFrame.loc[i] = [i,100 + treasuryCouponRate/2 ,tipsString, swapString, val, 100+treasuryCouponRate/2 ]
        else:
            print(discFact)
            print(intPolSwapPrice)
            val = treasuryCouponRate/2 -  intPolSwapPrice[i]                  
            stripsSum = stripsSum + val*discFact[i]
            tipsString  = str(tipsCouponRate/2)+'*I'+str(i)
            swapString = str(intPolSwapPrice[i])+ '-'+ tipsString
            finalFrame.loc[i] = [i,treasuryCouponRate/2 ,tipsString, swapString, val, treasuryCouponRate/2 ]
    finalFrame.loc[0] = [0,-treasuryPrice,-tipsPrice, 0, -stripsSum,  -(stripsSum +tipsPrice)]
    
    print("arbitrage :"+ str(treasuryPrice -(stripsSum +tipsPrice)))
    print(finalFrame)
    
    finalFrame.to_html('table.html')
if __name__=='__main__':
    main()

[ 0.          0.9991212   0.99905045  0.99502383  0.99024786  0.98622772
  0.97766324  0.96065669  0.94372544  0.93448171  0.92450206  0.90506628
  0.88230277  0.8636593   0.84701235  0.82917429  0.81227202  0.79854916
  0.78540168  0.76996978  0.75321844  0.73675052  0.72089565  0.70566515
  0.69107035  0.67712257  0.66383313  0.65121336  0.63927459  0.62802813
  0.61748531  0.60764756  0.59847671  0.5899247 ]
[ 0.          1.16511562  1.14442352  1.13233069  1.13156363  1.14219823
  1.15370778  1.15731108  1.16181146  1.176358    1.19389827  1.20666769
  1.22025443  1.24078911  1.26321947  1.28203468  1.30107266  1.32475212
  1.35046608  1.37472077  1.3975009   1.4194775   1.44058831  1.46058776
  1.47923026  1.49636369  1.51220973  1.52708347  1.54130005  1.55517457
  1.56902215  1.5831012   1.59744337  1.6120236 ]
[ 0.          0.9991212   0.99905045  0.99502383  0.99024786  0.98622772
  0.97766324  0.96065669  0.94372544  0.93448171  0.92450206  0.90506628
  0.88230277  0.8636593 

In [ ]:
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import math

tipsPrice =  2.375
df2 = pd.read_excel('C:/Users/hardik/Documents/HMGTstudies/Spring2018/Fixed Income/project/proto.xlsx', sheet_name='Sheet2')
df2.iloc[5,1]

# tempArr = [0,df2.iloc[1,1] ] 
# temp2 = [1,1,   df2.iloc[1,1]]
# temp2 =  temp2 + tempArr

# swapYears = df2.iloc[3,1:17]
# swapPeriods = [x * 2 for x in swapYears]
# swapRates = df2.iloc[5,1:17]
# inflationSwap = []
# for i in range(len(swapPeriods)):
#     inflationSwap.append(tipsPrice/2 * math.pow((1+swapRates[i]/200), swapPeriods[i]))

#print(inflationSwap)
# interpolatedPeriods = np.arange(2, 33 + 1, 1)
# intPolSwap = interp1d(swapPeriods, inflationSwap, kind='cubic')

# plt.plot(swapPeriods, inflationSwap, 'o', interpolatedPeriods, intPolSwap(interpolatedPeriods), '--')
# #>>> plt.legend(['data', 'linear', 'cubic'], loc='best')
# plt.show()
# print(intPolSwap(interpolatedPeriods))

